In [1]:
import pandas as pd
import numpy as np
import xlrd
from plotly.graph_objs import Scatter, layout
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import cufflinks as cf
from urllib.request import urlopen
import json
import re
cf.go_offline()###这两句是离线生成图片的设置
cf.set_config_file(offline=True, world_readable=True)
plotly.offline.init_notebook_mode(connected=True)

In [2]:
df1 = pd.read_csv('BCHAIN-MKPRU.csv')
df2 = pd.read_csv('LBMA-GOLD.csv')

In [9]:
df2 = df2.interpolate(method='linear', limit_direction='forward')

In [3]:
df1.head()

,Date,Value
0,9/11/16,621.65
1,9/12/16,609.67
2,9/13/16,610.92
3,9/14/16,608.82
4,9/15/16,610.38


In [4]:
df2.head()

,Date,USD (PM)
0,9/12/16,1324.60
1,9/13/16,1323.65
2,9/14/16,1321.75
3,9/15/16,1310.80
4,9/16/16,1308.35


In [10]:
data1 = df1['Value']
data2 = df2['USD (PM)']

In [6]:
from statsmodels.tsa.stattools import adfuller
print(adfuller(data1))
print(adfuller(df1['Value'].diff(1).fillna(0)))
print(adfuller(df1['Value'].diff(2).fillna(0)))

(-0.23774252687897476, 0.9338707177253291, 24, 1801, {'1%': -3.4339861159726306, '5%': -2.863146142843122, '10%': -2.5676250581282964}, 29168.935944417746)
(-8.537735567344068, 9.973933990343592e-14, 23, 1802, {'1%': -3.4339840952648695, '5%': -2.8631452508003057, '10%': -2.567624583142913}, 29166.993491682755)
(-7.6592535830552855, 1.7059734924782906e-11, 24, 1801, {'1%': -3.4339861159726306, '5%': -2.863146142843122, '10%': -2.5676250581282964}, 29212.440500169047)


In [11]:
print(adfuller(data2))
print(adfuller(df2['USD (PM)'].diff(1).fillna(0)))
print(adfuller(df2['USD (PM)'].diff(2).fillna(0)))

(-0.5079488932343523, 0.8904559666874652, 22, 1242, {'1%': -3.4356260602190356, '5%': -2.863869899335344, '10%': -2.5680104683371323}, 10021.010556670537)
(-7.957540057520856, 3.0090552229481678e-12, 21, 1243, {'1%': -3.435621806786881, '5%': -2.8638680226791444, '10%': -2.5680094689100477}, 10019.265250961456)
(-8.180001258817041, 8.179822010278522e-13, 21, 1243, {'1%': -3.435621806786881, '5%': -2.8638680226791444, '10%': -2.5680094689100477}, 10048.910459012992)


In [19]:
def f(x : str)->str:
    p = []
    for m in re.finditer('/', x):
        p.append(m.start())
    return "20" + x[-2:] + "-" + x[:p[0]].zfill(2) + "-" + x[p[0] + 1 : p[1]].zfill(2)
df2['date'] = df2['Date'].apply(lambda x : f(x))

In [21]:
df1['date'] = df1['Date'].apply(lambda x : f(x))

In [38]:
fig1 = go.Figure()
fig1.add_trace(Scatter(
    x=df1['date'],
    y=df1['Value'],
    legendgroup='group0',
    mode='lines',
    name='BCHAIN-MKPRU',
    marker=dict(color='MediumPurple')
))
fig1.update_layout(title={'text':'Time - Value Overview of Bitcoin','y' : 0.9,'x' : 0.5,  },
    xaxis_title='date',
    yaxis_title='U.S. dollars($)',
)
fig1.show()

In [32]:
fig2 = go.Figure()
fig2.add_trace(Scatter(
    x=df2['date'],
    y=df2['USD (PM)'],
    legendgroup='group0',
    mode='lines',
    name='LBMA-GOLD',
    marker=dict(color='Crimson')
))
fig2.update_layout(title={'text':'Time - Value Overview of Gold','y' : 0.9,'x' : 0.5,  },
    xaxis_title='date',
    yaxis_title='U.S. dollars($)',
)
fig2.show()

In [35]:
df2.isna().sum()

Date         0
USD (PM)    10
date         0
dtype: int64

In [44]:
missingPlace = (df2.isnull().values == True)
df2[missingPlace]

,Date,USD (PM),date
74,12/23/16,NaN,2016-12-23
77,12/30/16,NaN,2016-12-30
326,12/22/17,NaN,2017-12-22
329,12/29/17,NaN,2017-12-29
579,12/24/18,NaN,2018-12-24
582,12/31/18,NaN,2018-12-31
832,12/24/19,NaN,2019-12-24
835,12/31/19,NaN,2019-12-31
1086,12/24/20,NaN,2020-12-24
1089,12/31/20,NaN,2020-12-31


In [50]:
df2 = df2.interpolate(method='linear', limit_direction='forward')

In [53]:
fig2 = go.Figure()
fig2.add_trace(Scatter(
    x=df2['date'],
    y=df2['USD (PM)'],
    legendgroup='group0',
    mode='lines',
    name='LBMA-GOLD',
    marker=dict(color='Crimson')
))
fig2.update_layout(title={'text':'Time - Value Overview of Gold','y' : 0.9,'x' : 0.5,  },
    xaxis_title='date',
    yaxis_title='U.S. dollars($)',
)
fig2.show()

In [54]:
df1 = df1.drop('Date', axis=1)
df1.index = df1['date']

In [56]:
df1 = df1.drop('date', axis=1)

In [58]:
df2.index = df2['date']
df2 = df2.drop(['date', 'Date'], axis=1)

In [92]:
df = df1.join([df2], how='outer')

In [106]:
# 添加表示黄金是否售卖的特征, 1就是不能买
df['stopTrade'] = np.array(df['USD (PM)'].isnull().values == True, dtype=np.int)
df = df.fillna(method='ffill')

In [123]:
df['bitcoinMargin'] = np.append( 
    np.array([(df['Value'][i + 1] - df['Value'][i]) / df['Value'][i] for i in range(len(df['Value']) - 1)]),
    np.array([0])
)

In [124]:
df['bitcoinMargin'] = df['bitcoinMargin'].shift(1)

In [126]:
df['goldMargin'] = np.append( 
    np.array([(df['USD (PM)'][i + 1] - df['USD (PM)'][i]) / df['USD (PM)'][i] for i in range(len(df['USD (PM)']) - 1)]),
    np.array([0])
)
df['goldMargin'] = df['goldMargin'].shift(1)

In [131]:
np.mean(np.abs(df['goldMargin'].values[2:]))

0.0041929000740916055

In [133]:
np.mean(np.abs(df['bitcoinMargin'].values[1:]))

0.026655062733986674

In [134]:
df.shape

(1826, 5)

In [149]:
goldPrice, bitcoinPrice = df['USD (PM)'].values, df['Value'].values
df['bitcoinFive'] = np.append(
    np.array([np.mean(np.array(bitcoinPrice[i : i + 5])) for i in range(len(bitcoinPrice) - 4)]),
    np.array([0] * 4)
)
df['bitcoinFive'] = df['bitcoinFive'].shift(4)

In [154]:
df['goldTwenty'] = np.append(
    np.array([np.mean(np.array(goldPrice[i : i + 20])) for i in range(1, len(goldPrice) - 19)]),
    np.array([0] * 20)
)
df['goldTwenty'] = df['goldTwenty'].shift(20)

In [156]:
df.head()

,Value,USD (PM),stopTrade,bitcoinMargin,goldMargin,bitcoinFive,goldTwenty
2016-09-11,621.65,NaN,1,NaN,NaN,NaN,NaN
2016-09-12,609.67,1324.60,0,-0.019271,NaN,NaN,NaN
2016-09-13,610.92,1323.65,0,0.002050,-0.000717,NaN,NaN
2016-09-14,608.82,1321.75,0,-0.003437,-0.001435,NaN,NaN
2016-09-15,610.38,1310.80,0,0.002562,-0.008284,612.288,NaN


In [164]:
df.to_csv('data.csv')